### 4. 함수형 API를 사용해 복잡한 모델 만들기

In [1]:
import tensorflow as tf
from tensorflow import keras

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full,X_test,y_train_full,y_test = train_test_split(housing.data,housing.target)
X_train,X_valid,y_train,y_valid = train_test_split(X_train_full,y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [2]:
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation= "relu")(hidden1)
concat = keras.layers.concatenate([input_,hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_],outputs=[output])

In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 30)           270         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 30)           930         dense[0][0]                      
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 38)           0           input_1[0][0]                    
                                                                 dense_1[0][0]                

- 일부 특성은 짧은 경로로 전달하고 다른 특성들은 깊은 경로로 전달하는 경우

In [4]:
input_A = keras.layers.Input(shape=[5],name="wide_input")
input_B = keras.layers.Input(shape=[6],name="deep_input")
hidden1 = keras.layers.Dense(30, activation ="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation ="relu")(hidden1)
concat = keras.layers.concatenate([input_A,hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.Model(inputs=[input_A,input_B],outputs=[output])

In [5]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))
mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/20
363/363 [==============================] - 1s 981us/step - loss: 1.7854 - val_loss: 0.8606
Epoch 2/20
363/363 [==============================] - 0s 859us/step - loss: 0.8098 - val_loss: 0.7289
Epoch 3/20
363/363 [==============================] - 0s 955us/step - loss: 0.7085 - val_loss: 0.6698
Epoch 4/20
363/363 [==============================] - 0s 789us/step - loss: 0.6528 - val_loss: 0.6313
Epoch 5/20
363/363 [==============================] - 0s 909us/step - loss: 0.6147 - val_loss: 0.6024
Epoch 6/20
363/363 [==============================] - 0s 943us/step - loss: 0.5859 - val_loss: 0.5804
Epoch 7/20
363/363 [==============================] - 0s 850us/step - loss: 0.5636 - val_loss: 0.5628
Epoch 8/20
363/363 [==============================] - 0s 625us/step - loss: 0.5460 - val_loss: 0.5489
Epoch 9/20
363/363 [==============================] - 0s 697us/step - loss: 0.5326 - val_loss: 0.5378
Epoch 10/20
363/363 [==============================] - 0s 659us/step - loss: 0.521

- 규제를 위한 보조 출력 추가하기

In [6]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
model = keras.models.Model(inputs=[input_A, input_B],
                           outputs=[output, aux_output])

각 출력은 자신만의 손실 함수가 필요함. 따라서 모델을 컴파일할 때 손실의 리스트를 전달해야 한다.(하나의 손실을 전달하면 케라스는  
    모든 출력의 손실함수가 동일하다고 가정한다.)  
기본적으로 케라스는 나열된 손실을 모두 더하여 최종 손실을 구해 훈련에 사용한다.  
보조 출력보다 주 출력에 더 관심이 많다면(보조 출력은 규제로만 사용되므로), 주 출력의 손실에 더 많은 가중치를 부여해야 한다.  

In [7]:
model.compile(loss=["mse","mse"],loss_weights=[0.9,0.1],optimizer="sgd")

모델을 훈련할 때 각 출력에 대한 레이블을 제공해야 한다. 여기에서는 주 출력과 보조출력이 같은 것을 예측해야 하므로 동일한  
레이블을 사용한다. 따라서 y_train 대신에 (y_train,y_train)을 전달한다(y_valid와 y_test도 동일하다.)

In [8]:
history = model.fit([X_train_A,X_train_B], [y_train,y_train], epochs=20,
                    validation_data=([X_valid_A,X_valid_B],[y_valid,y_valid]))

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 1.0127 - main_output_loss: 0.9249 - aux_output_loss: 1.8033 - val_loss: 0.6360 - val_main_output_loss: 0.5817 - val_aux_output_loss: 1.1247
Epoch 2/20
363/363 [==============================] - 0s 796us/step - loss: 0.6025 - main_output_loss: 0.5487 - aux_output_loss: 1.0868 - val_loss: 0.6793 - val_main_output_loss: 0.6494 - val_aux_output_loss: 0.9485
Epoch 3/20
363/363 [==============================] - 0s 830us/step - loss: 0.5563 - main_output_loss: 0.5157 - aux_output_loss: 0.9217 - val_loss: 0.5322 - val_main_output_loss: 0.4983 - val_aux_output_loss: 0.8375
Epoch 4/20
363/363 [==============================] - 0s 791us/step - loss: 0.5105 - main_output_loss: 0.4754 - aux_output_loss: 0.8268 - val_loss: 0.5097 - val_main_output_loss: 0.4816 - val_aux_output_loss: 0.7624
Epoch 5/20
363/363 [==============================] - 0s 789us/step - loss: 0.5316 - main_output_loss: 0.5048 - aux_output_loss: 0.7733 - 

모델을 평가하면 케라스는 개별 손실과 함께 총 손실을 반환한다.

In [9]:
total_loss, main_loss, aux_loss = model.evaluate(
    [X_test_A, X_test_B], [y_test, y_test])

162/162 [==============================] - 0s 546us/step - loss: 0.3829 - main_output_loss: 0.3705 - aux_output_loss: 0.4950


In [10]:
y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])

여기에서 보는 것처럼 함수형 API는 원하는 어떤 종류의 구조도 손쉽게 만들 수 있다.

 ### 5.서브클래싱 API로 동적 모델 만들기
 시퀸셜 API와 함수형 API는 모두 선언적이다. 사용할 층과 연결 방식을 먼저 정의해야 한다. 그 다음 모델에 데이터를 주입하여 훈련이나 추론을 시작할 수 있다. 
 하지만, 어떤 모델은 반복문을 포함하고 다양한 크기를 다루어야 하며 조건문을 가지는 등 여러 가지 동적인 구조를 필요로 한다.
 이런 경우에 조금 더 명령형 프로그래밍 스타일이 필요하다면 서브클래싱API가 정답이다.

- 서브 클래싱 api  
    간단히 Model 클래스를 상속한 다음 생성자 안에서 필요한 층을 만든다. 그 다음 call() 메서드 안에 수행하려는 연산을 기술한다.  
    예를 들어 다음 WideAndDeepModel 클래스의 인스턴스는 앞서 함수형 API로 만든 모델과 동일한 기능을 수행한다.  
    이전에 했던 것처럼 이 인스턴스를 사용해 모델 컴파일,훈련,평가,예측을 수행할 수 있다.

In [11]:
class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs) # 표준 매개변수를 처리한다(예를 들면, name)
        self.hidden1= keras.layers.Dense(units, activation=activation)
        self.hidden2= keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
    
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1= self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A,hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output
    
model = WideAndDeepModel()        

### 6. 모델 저장과 복원

시퀸셜 api와 함수형 api를 사용하면 훈련된 케라스 모델을 저장하는 것은 다음처럼 매우 쉽다.

In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])    

In [13]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)

Epoch 1/10
363/363 [==============================] - 1s 826us/step - loss: 2.0527 - val_loss: 0.7422
Epoch 2/10
363/363 [==============================] - 0s 550us/step - loss: 0.6884 - val_loss: 0.6451
Epoch 3/10
363/363 [==============================] - 0s 594us/step - loss: 0.6278 - val_loss: 0.6178
Epoch 4/10
363/363 [==============================] - 0s 595us/step - loss: 0.5991 - val_loss: 0.5964
Epoch 5/10
363/363 [==============================] - 0s 556us/step - loss: 0.5779 - val_loss: 0.5806
Epoch 6/10
363/363 [==============================] - 0s 526us/step - loss: 0.5605 - val_loss: 0.5656
Epoch 7/10
363/363 [==============================] - 0s 642us/step - loss: 0.5458 - val_loss: 0.5529
Epoch 8/10
363/363 [==============================] - 0s 590us/step - loss: 0.5332 - val_loss: 0.5415
Epoch 9/10
363/363 [==============================] - 0s 663us/step - loss: 0.5216 - val_loss: 0.5311
Epoch 10/10
162/162 [==============================] - 0s 461us/step - loss: 0.520

In [14]:
from tensorflow.keras.models import load_model
model.save("my_keras_model.h5")

훈련이 몇 시간 동안 지속되는 경우 훈련 마지막에 모델을 저장하는 것 뿐만이 아니라 훈련 도중 일정 간격으로 체크포인트를 저장해야 한다.  
fit() 메서드에서 체크포인트를 저장하는 방법은 콜백을 사용하면 된다.

In [16]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

X_new = X_test[:3]

In [17]:
model = keras.models.load_model("my_keras_model.h5")
model.predict(X_new)

array([[0.5764959],
       [1.2645893],
       [3.3101127]], dtype=float32)

### 7. 콜백 사용하기
fit()메서드의 callbacks 매개변수를 사용하여 케라스가 훈련의 시작이나 끝에 호출할 객체 리스트를 지정할 수 있다. 또는 에포크의  
시작이나 끝, 각 배치 처리 전후에 호출할 수도 있다. 또는 에포크의 시작이나 끝, 각 배치 처리 전후에 호출할 수도 있다.  
예를 들어 ModelCheckpoint는 훈련하는 동안 일정한 간격으로 모델의 체크포인트를 저장한다. 기본적으로 매 에포크의 끝에서 호출된다.

In [18]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])    

훈련하는 동안 검증 세트를 사용하면 ModelChecpoint를 만들 때 save_best_only=True로 지정할 수 있다.  
이렇게 하면 최상의 검증 세트 점수에서만 모델을 저장한다. 오랜 훈련시간으로 훈련세트에 과대적합될 걱정을 하지 않아도 된다.  
훈련이 끝난 후 마지막에 저장된 모델을 복원하면 된다. 그 모델이 검증 세트에서 최상의 점수를 낸 모델이다.  
다음 코드는 조기 종료를 구현하는 방법이다.

In [20]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb])
model = keras.models.load_model("my_keras_model.h5") # 최상의 모델로 롤백
mse_test = model.evaluate(X_test, y_test)

Epoch 1/10
363/363 [==============================] - 0s 782us/step - loss: 0.4630 - val_loss: 0.4270
Epoch 2/10
363/363 [==============================] - 0s 545us/step - loss: 0.4499 - val_loss: 0.4147
Epoch 3/10
363/363 [==============================] - 0s 533us/step - loss: 0.4391 - val_loss: 0.4084
Epoch 4/10
363/363 [==============================] - 0s 590us/step - loss: 0.4306 - val_loss: 0.4033
Epoch 5/10
363/363 [==============================] - 0s 581us/step - loss: 0.4237 - val_loss: 0.4014
Epoch 6/10
363/363 [==============================] - 0s 629us/step - loss: 0.4174 - val_loss: 0.4271
Epoch 7/10
363/363 [==============================] - 0s 664us/step - loss: 0.4125 - val_loss: 0.3956
Epoch 8/10
363/363 [==============================] - 0s 611us/step - loss: 0.4081 - val_loss: 0.3834
Epoch 9/10
363/363 [==============================] - 0s 579us/step - loss: 0.4041 - val_loss: 0.4004
Epoch 10/10
162/162 [==============================] - 0s 375us/step - loss: 0.402

조기 종료를 구현하는 또 다른 방법은 EarlyStopping 콜백을 사용하는 것입니다. 일정 에포크(patience 매개변수로 지정한다)동안 검증 세트에 대한 점수가 향상되지 않으면 훈련을 멈춘다. 선택적으로 최상의 모델을 복원할 수도 있다. 
컴퓨터가 문제를 일으키는 경우를 대비해서 체크포인트 저장 콜백과 시간과 컴퓨팅 자원을 낭비하지 않기 위해 진전이 없는 경우 훈련을  
일찍 멈추는 콜백을 함께 사용할 수 있다.

In [21]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

In [22]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, early_stopping_cb])
mse_test = model.evaluate(X_test, y_test)

Epoch 1/100
363/363 [==============================] - 1s 1ms/step - loss: 0.4039 - val_loss: 0.4103
Epoch 2/100
363/363 [==============================] - 0s 532us/step - loss: 0.4008 - val_loss: 0.3884
Epoch 3/100
363/363 [==============================] - 0s 560us/step - loss: 0.3979 - val_loss: 0.3738
Epoch 4/100
363/363 [==============================] - 0s 608us/step - loss: 0.3953 - val_loss: 0.3858
Epoch 5/100
363/363 [==============================] - 0s 597us/step - loss: 0.3926 - val_loss: 0.4135
Epoch 6/100
363/363 [==============================] - 0s 600us/step - loss: 0.3907 - val_loss: 0.3937
Epoch 7/100
363/363 [==============================] - 0s 580us/step - loss: 0.3884 - val_loss: 0.3910
Epoch 8/100
363/363 [==============================] - 0s 650us/step - loss: 0.3864 - val_loss: 0.4155
Epoch 9/100
363/363 [==============================] - 0s 577us/step - loss: 0.3850 - val_loss: 0.3698
Epoch 10/100
363/363 [==============================] - 0s 600us/step - los

- 사용자 정의 콜백 만들기  
    다음의 콜백은 훈련하는 동안 검증 손실과 훈련손실의 비율을 출력한다.(즉 과대적합을 감지한다.)

In [24]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))

In [25]:
val_train_ratio_cb = PrintValTrainRatioCallback()
history = model.fit(X_train, y_train, epochs=1,
                    validation_data=(X_valid, y_valid),
                    callbacks=[val_train_ratio_cb])

363/363 [==============================] - 0s 682us/step - loss: 0.3437 - val_loss: 0.3918

val/train: 1.14


### 8. 텐서보드를 사용해 시각화하기
    텐서보드는 훈련 동안 학습 곡선을 그리거나 여러 실행 간의 학습 곡선을 비교하고 계싼 그래프 시각화와 훈련 통계 분석을 수행할 수 있다.  
    또한 모델이 생성한 이미지를 확인하거나 3D에 투영된 복잡한 다차원 데이터를 시각화하고 자동으로 클러스터링을 해주는 등 많은 기능을 제공한다. 텐서보드는 텐서플로를 설치할 때 자동으로 설치되므로 이미 시스템에 준비되어 있다.
    텐서보드를 사용하려면 프로그램을 수저앟여 이벤트파일event file 이라는 특별한 이진 로그 파일에 시각화 하려는 데이터를 출력해야 한다.  
    각각의 이진 데이터 레코드를 서미리라고 부른다. 
    텐서보드 서버는 로그 디렉터리를 모니터링하고 자동으로 변경사항을 읽어 그래프를 업데이터 한다. 훈련하는 중가넹 학습 곡선 같이 실시간 데이터를 시각화 할 수 있다.  
    일반적으로 텐서보드 서버가 루트root 로그 데렉토리를 가리키고 프로글매은 실행할 때마다 다른 서브디렉터리에 이벤트를 기록한다. 
    이렇게 하면 복잡하지 않게 하나의 텐서보드 서버가 여러 번 실행한 프로그램의 결과를 시각화하고 비교할 수 있다.
    먼저 텐서보드 로그를 위해 사용할 루트 로그 디렉터리를 정의하겟다. 현재 날짜와 시간을 사용해 실행할 때마다 다른 서브디렉터리 경로를 생성하는 간단한 함수도 만들겠다. 테스트하는 하이퍼라미터 갑소가 같은 추가적인 정보를 로그 디렉터리 이름으로 사용할 수 있다.  
    이렇게 하면 텐서보드에서 어던 로그인지 구분하기 편하다.

In [26]:
import os
root_logdir = os.path.join(os.curdir, "my_logs")

In [27]:
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
run_logdir

'.\\my_logs\\run_2021_09_14-17_22_01'

In [28]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])    
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

In [29]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, tensorboard_cb])

Epoch 1/30
363/363 [==============================] - 1s 999us/step - loss: 2.4710 - val_loss: 2.2079
Epoch 2/30
363/363 [==============================] - 0s 749us/step - loss: 0.8573 - val_loss: 0.8581
Epoch 3/30
363/363 [==============================] - 0s 580us/step - loss: 0.6642 - val_loss: 0.6215
Epoch 4/30
363/363 [==============================] - 0s 815us/step - loss: 0.6112 - val_loss: 0.5645
Epoch 5/30
363/363 [==============================] - 0s 752us/step - loss: 0.5759 - val_loss: 0.5337
Epoch 6/30
363/363 [==============================] - 0s 699us/step - loss: 0.5465 - val_loss: 0.5055
Epoch 7/30
363/363 [==============================] - 0s 637us/step - loss: 0.5216 - val_loss: 0.4860
Epoch 8/30
363/363 [==============================] - 0s 751us/step - loss: 0.5005 - val_loss: 0.4638
Epoch 9/30
363/363 [==============================] - 0s 628us/step - loss: 0.4831 - val_loss: 0.4476
Epoch 10/30
363/363 [==============================] - 0s 828us/step - loss: 0.468